# Predictive Modeling of County-Level Acute Food Insecurity in Kenya

## Business Understanding

### Context and Motivation

Food insecurity remains a persistent challenge in Kenya, particularly in arid and semi-arid lands (ASALs). According to the [World Food Programme – Kenya](https://www.wfp.org/countries/kenya), **approximately 4.3 million people were projected to face acute food insecurity during the peak of the 2023 drought**, highlighting the scale of climate-driven vulnerability. The [Global Hunger Index – Kenya](https://www.globalhungerindex.org/kenya.html) classifies Kenya’s hunger level as **“serious”**, indicating ongoing structural food security challenges. Projections from the [EU Knowledge for Policy platform](https://knowledge4policy.ec.europa.eu/publication/kenya-acute-food-insecurity-situation-july-september-2024-projection-october-2024_en) further showed that **multiple counties were expected to experience IPC Phase 3 (Crisis) or worse during seasonal periods in 2024**.

These recent assessments demonstrate that large populations in Kenya repeatedly face acute food insecurity. However, current tools such as IPC primarily describe conditions after they occur. This limits early action and often leads to reactive humanitarian response. There is therefore a clear need for predictive systems that can anticipate food insecurity and support proactive decision-making.

---

### Project Goal

This project develops a predictive early-warning model for acute food insecurity risk at the county level in Kenya by integrating historical IPC classifications with key drivers including climate conditions, staple food price dynamics, conflict events, and baseline poverty indicators. The resulting model and dashboard will provide forward-looking risk scores to support early action by county governments, humanitarian organizations, and policy planners, helping shift decision-making from reactive response to anticipatory planning.

---

### Project Objectives

1. Build a county-level early-warning system that anticipates IPC Phase 3+ food insecurity 1–3 months ahead.
2. Reveal the most influential climate, market, conflict, and poverty drivers shaping food insecurity risk across Kenya.
3. Produce reliable risk scores and hotspot signals to support timely policy and humanitarian action.
4. Deliver a decision-ready dashboard that visualizes risk trends, spatial patterns, and key contributing factors.

---

## Data Understanding

This project leverages five key data sources that represent both the target outcome and predictive signals:

### IPC Kenya Classifications

The IPC dataset provides historical classifications of food insecurity severity (Phase 1–5) by county and period. It serves as the **target variable** for supervised learning by flagging whether a county is in crisis (Phase 3 or worse) at a given time.

### Subnational Rainfall Indicators

Rainfall data contains monthly totals, historical averages, and anomalies for Kenyan counties. These indicators capture **climate stress**, particularly drought conditions, which are a known driver of crop failure and livestock losses.

### WFP Food Prices

Food price data tracks the cost of staple commodities — such as maize, beans, and rice — in various markets over time. Changes in staple food prices reflect **market stress** and reduced food access for net buyers, especially among low-income households.

### Political Violence Events

Conflict event data records the number of violent events and associated fatalities by county and month. Political violence disrupts markets, displacement, and access to humanitarian support, making it a critical **shock indicator** for food insecurity modeling.

### Multidimensional Poverty Index (MPI)

The MPI dataset provides county-level socioeconomic vulnerability measures, including headcount poverty rates and intensity of deprivation. These indicators act as **baseline contextual features**, helping the model account for structural vulnerability that can exacerbate the impact of shocks.